In [1]:
import os
import gzip
import json
from collections import defaultdict, OrderedDict

import pandas as pd
import spacy
import tokenize_uk

from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
from sklearn.neighbors import KNeighborsClassifier

In [7]:
!python -m spacy init-model uk /tmp/uk_vectors --vectors-loc ../../../../news.cased.tokenized.word2vec.300d

✔ Successfully created model
365319it [00:28, 12794.20it/s]./../news.cased.tokenized.word2vec.300d
✔ Loaded vectors from
../../../../news.cased.tokenized.word2vec.300d
✔ Sucessfully compiled vocab
365495 entries, 365319 vectors


In [2]:
nlp_uk = spacy.load('/tmp/uk_vectors')

In [3]:
base_dir = '../../../../1551.gov.ua/raw/'

file_list = []
for letter in os.listdir(base_dir):
    for filename in os.listdir(base_dir + letter):
        file_list.append(base_dir + letter + '/' + filename)

call_data = []

for filename in file_list:
    with gzip.open(filename, 'r') as zipfile:
        call_info = json.load(zipfile)
        call_data.append((call_info[0]['CallZText'], call_info[0]['CallZType']))

'Кількість даних і приклад', len(call_data), call_data[0]

('Кількість даних і приклад',
 127329,
 ('Хочу подякувати головному інженеру ТОВ ”Житловик Плюс” Омельченку О.А. та колективу товариства за якісну роботу.',
  'Подяки головному інженеру ЖЕКу'))

In [4]:
categories = set([c[1] for c in call_data])
'Кількість унікальних категорій {}'.format(len(categories))

'Кількість унікальних категорій 920'

In [5]:
call_df = pd.DataFrame(call_data)
call_df.columns = ['text', 'subcategory']
call_df

,text,subcategory
0,Хочу подякувати головному інженеру ТОВ ”Житлов...,Подяки головному інженеру ЖЕКу
1,Почему опять нет горячей воды? Я видел что за ...,Відсутність ГВП
2,"не має гарячої води, замість гарячої - холодна...",Відсутність ГВП
3,На вулиці Байковій присутня дошка з інформаціє...,Встановлення меморіальних дощок видатним діяча...
4,Добрый день! По улице Жуковского возле дома №...,Облаштування та технічний стан бюветного компл...
...,...,...
127324,"23 августа, аварийная служба киевэнерго работа...",Відсутність ГВП
127325,Знову відсутнє гаряче водопостачання! Інформац...,Відсутність ГВП
127326,Другий тиждень відсутнє ГВП в будинку 43/3.Не...,Відсутність ГВП
127327,Відсутній тиск холодного та гарячого водопоста...,Відсутній тиск ХВП до 9-го поверху включно (”К...


In [6]:
call_df.to_csv('raw_call_data.csv')
#call_df = pd.read_csv('raw_call_data.csv')

In [7]:
count_df = call_df.groupby('subcategory').count()
count_df.query('text == 1')

,text
subcategory,
@ Зняття дерев без застосування автопідіймача,1
@ Плата для заїзду автотранспорту на кладовища міста,1
@ Подяки за роботу КО ”Київзеленбуд”,1
@ Формована обрізка кущів,1
Індивідуальне будівництво,1
...,...
Створення та діяльність садових товариств,1
Утримання об’єктів комунальної власності м. Києва (КМДА і Будинок профспілок),1
Харчування пацієнтів психоневрологічних диспансерів і лікарень,1


Частину цих одиниць я спробую згрупувати з иншими категоріями, а ті, що не вийде, підуть під ніж

## Проведемо чистку даних

In [15]:
from category_changes import clean_call_data

cleaned_call_data = clean_call_data(call_data)
cleaned_categories = set([c[2] for c in cleaned_call_data])
print('Кількість унікальних категорій: {}'.format(len(cleaned_categories)))
call_df = pd.DataFrame(cleaned_call_data)
call_df.columns = ['text', 'category', 'new_category']
call_df

Кількість унікальних категорій: 760


,text,category,new_category
0,Хочу подякувати головному інженеру ТОВ ”Житлов...,Подяки головному інженеру ЖЕКу,Подяки
1,Почему опять нет горячей воды? Я видел что за ...,Відсутність ГВП,Відсутність ГВП
2,"не має гарячої води, замість гарячої - холодна...",Відсутність ГВП,Відсутність ГВП
3,На вулиці Байковій присутня дошка з інформаціє...,Встановлення меморіальних дощок видатним діяча...,Встановлення меморіальних дощок видатним діяча...
4,Добрый день! По улице Жуковского возле дома №...,Облаштування та технічний стан бюветного компл...,Облаштування та технічний стан бюветного компл...
...,...,...,...
127210,"23 августа, аварийная служба киевэнерго работа...",Відсутність ГВП,Відсутність ГВП
127211,Знову відсутнє гаряче водопостачання! Інформац...,Відсутність ГВП,Відсутність ГВП
127212,Другий тиждень відсутнє ГВП в будинку 43/3.Не...,Відсутність ГВП,Відсутність ГВП
127213,Відсутній тиск холодного та гарячого водопоста...,Відсутній тиск ХВП до 9-го поверху включно (”К...,Відсутній тиск ХВП до 9-го поверху включно (”К...


In [9]:
call_df.to_csv('call_data.csv')

## Baseline

In [11]:
def vectorize(text):
    return(nlp_uk(text).vector)

copy_df = call_df.copy()

y_column = 'new_category'
y = copy_df.loc[:, y_column]

X_train, X_test, y_train, y_test = train_test_split(
    list(copy_df.to_dict()['text'].values()), y,
    train_size=0.8, test_size=0.2, random_state=0, shuffle=True, stratify=y
)

train_vectorized = []
for row in X_train:
    train_vectorized.append(vectorize(row))
test_vectorized = []
for row in X_test:
    test_vectorized.append(vectorize(row))

In [12]:
neigh = KNeighborsClassifier(n_neighbors=5, metric='cosine')
neigh.fit(train_vectorized, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='cosine',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [13]:
y_pred = neigh.predict(test_vectorized)

print(classification_report(y_test.to_list(), list(y_pred)))

/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                                                                                  precision    recall  f1-score   support

                                                                  @ Зняття дерев       0.08      1.00      0.15         1
                                                               @ Корчування пнів       0.00      0.00      0.00         2
                                                Інтервал руху міської електрички       0.00      0.00      0.00         4
                                    Інформаційне забезпечення салонів транспорту       0.29      0.80      0.42         5
                                                                     Інші Подяки       0.18      0.26      0.21       103
              Інші види соціальної допомоги, що не ввійшли  до типового переліку       0.00      0.00      0.00         8
                                     Інші питання діяльності релігійних закладів       0.40      0.50      0.44         4
                       

## Кластеризація текстів

In [38]:
kmeans = KMeans(n_clusters=50)
kmeans.fit(train_vectorized)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=50, n_init=10, n_jobs=None, precompute_distances='auto',
       random_state=None, tol=0.0001, verbose=0)

In [39]:
labeled_categories = defaultdict(list)

for index, category in enumerate(cleaned_categories):
    label = kmeans.labels_[index]
    labeled_categories[label].append(category)

min_cats = 9999
max_cats = 0
cat_to_labels = {}
cat_to_labels_count = defaultdict(int)
for label, categories in labeled_categories.items():
    if len(categories) > max_cats:
        max_cats = len(categories)
    if len(categories) < min_cats:
        min_cats = len(categories)

    for category in categories:
        cat_to_labels_count[category] += 1
        cat_to_labels[category] = label

print(f'Cluster has from {min_cats} to {max_cats} categories')
for category, label_count in cat_to_labels_count.items():
    if label_count > 1:
        print(f'{category} has {label_count} labels')

Cluster has from 2 to 66 categories


In [40]:
label_test = []
for category in y_test.to_list():
    label_test.append(cat_to_labels[category])
label_pred = kmeans.predict(test_vectorized)
print(classification_report(label_test, label_pred))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         2
           1       0.01      0.01      0.01        86
           2       0.15      0.05      0.07      4361
           3       0.01      0.03      0.01       148
           4       0.00      0.00      0.00       110
           5       0.01      0.02      0.01       100
           6       0.00      0.00      0.00         8
           7       0.02      0.02      0.02       353
           8       0.02      0.02      0.02       657
           9       0.02      0.01      0.01       595
          10       0.11      0.10      0.11      1102
          11       0.04      0.07      0.05       664
          12       0.00      0.00      0.00         0
          13       0.01      0.00      0.00       856
          14       0.00      0.00      0.00        20
          15       0.00      0.00      0.00        22
          16       0.04      0.05      0.04       316
          17       0.00    

/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Я не став тут робити багато замірів, результат стабільно поганий, якщо кластеризувати по тексту:

```
Вгадування кластеру
   кластери        macro-avg
    10             0.10
    20             0.05
    50             0.02
```

Такий результат є непридатний :(

## Кластеризація по категорії

Як на мене трохи беззмістовне заняття, адже неможливо чітко вгадати категорію. Але я провів цей експеримент просто помилившись

In [42]:
categories_vectorized = []
for category in cleaned_categories:
    categories_vectorized.append(vectorize(category))

len(categories_vectorized)

760

In [43]:
kmeans = KMeans(n_clusters=30)
kmeans.fit(categories_vectorized)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=30, n_init=10, n_jobs=None, precompute_distances='auto',
       random_state=None, tol=0.0001, verbose=0)

In [44]:
labeled_categories = defaultdict(list)

for index, category in enumerate(cleaned_categories):
    label = kmeans.labels_[index]
    labeled_categories[label].append(category)

min_cats = 9999
max_cats = 0
for label, categories in labeled_categories.items():
    if len(categories) > max_cats:
        max_cats = len(categories)
    if len(categories) < min_cats:
        min_cats = len(categories)

print(f'Cluster has from {min_cats} to {max_cats} categories')

category_to_label = {}
for label, categories in labeled_categories.items():
    for category in categories:
        category_to_label[category] = label

labeled_call_data = []
for call in cleaned_call_data:
    text, category, new_category = call
    label = category_to_label[new_category]
    labeled_call_data.append((text, category, new_category, label))

call_df = pd.DataFrame(labeled_call_data)
call_df.columns = ['text', 'category', 'new_category', 'label']
call_df

Cluster has from 5 to 97 categories


,text,category,new_category,label
0,Хочу подякувати головному інженеру ТОВ ”Житлов...,Подяки головному інженеру ЖЕКу,Подяки,15
1,Почему опять нет горячей воды? Я видел что за ...,Відсутність ГВП,Відсутність ГВП,15
2,"не має гарячої води, замість гарячої - холодна...",Відсутність ГВП,Відсутність ГВП,15
3,На вулиці Байковій присутня дошка з інформаціє...,Встановлення меморіальних дощок видатним діяча...,Встановлення меморіальних дощок видатним діяча...,15
4,Добрый день! По улице Жуковского возле дома №...,Облаштування та технічний стан бюветного компл...,Облаштування та технічний стан бюветного компл...,7
...,...,...,...,...
127210,"23 августа, аварийная служба киевэнерго работа...",Відсутність ГВП,Відсутність ГВП,15
127211,Знову відсутнє гаряче водопостачання! Інформац...,Відсутність ГВП,Відсутність ГВП,15
127212,Другий тиждень відсутнє ГВП в будинку 43/3.Не...,Відсутність ГВП,Відсутність ГВП,15
127213,Відсутній тиск холодного та гарячого водопоста...,Відсутній тиск ХВП до 9-го поверху включно (”К...,Відсутній тиск ХВП до 9-го поверху включно (”К...,15


In [45]:
count_df = call_df.groupby('label').count()
count_df.query('text == 1')

,text,category,new_category
label,,,


## Покластерне передбачення

In [46]:
def cluster_train_test_split(orig_df, cluster_column, y_column):
    result = {}
    df = orig_df.copy()

    for i in df[cluster_column].unique():
        #print(f'Working on cluster {i}')
        df_i = df.loc[df[cluster_column] == i]
        y_i = df_i.loc[:, y_column]
        #df_i.drop([cluster_column, y_column], axis=1, inplace=True)
        X_i_train, X_i_test, y_i_train, y_i_test = train_test_split(
            list(df_i.to_dict()['text'].values()), y_i,
            train_size=0.7, test_size=0.3, random_state=0, shuffle=True, stratify=y_i
        )
        train_i_vectorized = []
        for row in X_i_train:
            train_i_vectorized.append(vectorize(row))

        test_i_vectorized = []
        for row in X_i_test:
            test_i_vectorized.append(vectorize(row))

        result[i] = (train_i_vectorized, test_i_vectorized, y_i_train, y_i_test)

    return result

cluster_train_test = cluster_train_test_split(call_df, 'label', 'new_category')

In [47]:
all_y_test = []
all_y_pred = []

for cluster, cluster_data in cluster_train_test.items():
    train_data, test_data, y_train, y_test = cluster_data

    neigh = KNeighborsClassifier(n_neighbors=min(5, len(test_data)), metric='cosine')
    neigh.fit(train_data, y_train)

    y_pred = neigh.predict(test_data)

    all_y_test.extend(y_test.to_list())
    all_y_pred.extend(list(y_pred))

In [48]:
print(classification_report(all_y_test, all_y_pred))

/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                                                                                  precision    recall  f1-score   support

                                              @ Викошування газонів мотокосаркою       0.00      0.00      0.00         1
                                                                  @ Зняття дерев       0.13      1.00      0.24         2
                                                               @ Корчування пнів       0.00      0.00      0.00         4
                                                Інтервал руху міської електрички       0.00      0.00      0.00         5
                                    Інформаційне забезпечення салонів транспорту       1.00      0.88      0.93         8
                                                                     Інші Подяки       0.86      0.60      0.70       154
              Інші види соціальної допомоги, що не ввійшли  до типового переліку       0.22      0.17      0.19        12
                       

macro average покращився вдвічі в порівнянні з baseline'ом. Але, нажаль, цю модель неможливо використати для передбачення.